<br>

# Book Recommendation System

<br>

<br>

#### Importing Required Libraries

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import pickle

#### Reading Data

In [5]:
path = 'C:\\Users\\piyus\\GoogleDrive\\Data_Science\\Projects\\Recommendation System\\User Based\\Book_Recommendation_System\\Data\\'

books = pd.read_csv(path+'Books.csv')
ratings = pd.read_csv(path + 'Ratings.csv')
users = pd.read_csv(path + 'Users.csv')

In [6]:
display(books.head(), ratings.head(), users.head())

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
display(books.shape, ratings.shape, users.shape)

(271360, 8)

(1149780, 3)

(278858, 3)

**books**

- 1. ISBN :- Unique Number given to every book.



**ratings**

- 1. Book-Rating :- Rating Out of 10

#### Lowering Column Names

In [8]:
books.columns = books.columns.str.lower()

In [9]:
ratings.columns = ratings.columns.str.lower()

In [10]:
users.columns = users.columns.str.lower()

#### Rnaming Columns

In [11]:
books.rename(columns = {'book-title' : 'book_title', 
                        'book-author' : 'book_author',
                        'year-of-publication' : 'year_of_publication',
                        'image-url-s' : 'image_url_s',
                        'image-url-m' : 'image_url_m',
                        'image-url-l' : 'image_url_l'},
               inplace = True
            )

In [12]:
ratings.rename(columns = {'user-id' : 'user_id',
                          'book-rating' : 'book_rating'},
               inplace = True
              )

In [13]:
users.rename(columns = {'user-id' : 'user_id'},
             inplace = True
            )

#### Handling For Missing

In [14]:
books.isna().sum()

isbn                   0
book_title             0
book_author            1
year_of_publication    0
publisher              2
image_url_s            0
image_url_m            0
image_url_l            3
dtype: int64

In [15]:
# books.dropna(inplace = True)

In [16]:
ratings.isna().sum()

user_id        0
isbn           0
book_rating    0
dtype: int64

In [17]:
users.isna().sum()

user_id          0
location         0
age         110762
dtype: int64

#### Looking for Duplicates

- **Every Book is Unique.**

In [18]:
books.loc[books['isbn'].duplicated(), 'isbn'].value_counts()

Series([], Name: isbn, dtype: int64)

- **Single Book can be rated by multiple uasers.**

In [19]:
ratings.loc[ratings['isbn'].duplicated(), 'isbn'].value_counts()

0971880107    2501
0316666343    1294
0385504209     882
0060928336     731
0312195516     722
              ... 
0863076645       1
0375757538       1
0722155999       1
3423129867       1
0380796155       1
Name: isbn, Length: 143511, dtype: int64

In [20]:
users.loc[ratings['user_id'].duplicated(), 'user_id'].value_counts()

5         1
185013    1
185018    1
185019    1
185020    1
         ..
92845     1
92846     1
92847     1
92848     1
278858    1
Name: user_id, Length: 253063, dtype: int64

<br>

<br>

### Popularity Based Recomendation

In [21]:
df = pd.merge(books, ratings, on = 'isbn', how = 'inner').drop(columns = ['isbn', 'publisher', 'image_url_s', 'image_url_l'])
df

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
0,Classical Mythology,Mark P. O. Morford,2002,http://images.amazon.com/images/P/0195153448.0...,2,0
1,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,8,5
2,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,41385,0
...,...,...,...,...,...,...
1031131,There's a Bat in Bunk Five,Paula Danziger,1988,http://images.amazon.com/images/P/0440400988.0...,276463,7
1031132,From One to One Hundred,Teri Sloat,1991,http://images.amazon.com/images/P/0525447644.0...,276579,4
1031133,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,http://images.amazon.com/images/P/006008667X.0...,276680,0
1031134,Republic (World's Classics),Plato,1996,http://images.amazon.com/images/P/0192126040.0...,276680,0


- **Number of times book is rated > 200**

In [22]:
df.groupby(['book_title'])['book_rating'].count()

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    4
 Always Have Popsicles                                                                                        1
 Apple Magic (The Collector's series)                                                                         1
 Ask Lily (Young Women of Faith: Lily Series, Book 5)                                                         1
 Beyond IBM: Leadership Marketing and Finance for the 1990s                                                   1
                                                                                                             ..
Ã?Â?lpiraten.                                                                                                 2
Ã?Â?rger mit Produkt X. Roman.                                                                                4
Ã?Â?sterlich leben.                                                                          

In [23]:
count_rating_title = df.groupby(['book_title'])['book_rating'].count().reset_index().rename(columns =
                                                                                            {'book_rating' : 'no_of_ratings'})
count_rating_title

,book_title,no_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [24]:
count_rating_title[count_rating_title['no_of_ratings'] > 200]

,book_title,no_of_ratings
764,1984,284
818,1st to Die: A Novel,509
1048,2nd Chance,356
1760,A Bend in the Road,346
2120,A Case of Need,236
...,...,...
234951,Wild Animus,2502
236334,Wish You Well,234
236724,Without Remorse,203
240413,Zen and the Art of Motorcycle Maintenance: An ...,210


- **Average rating for top book 50**

In [25]:
df.groupby(['book_title'])['book_rating'].mean()

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    2.250000
 Always Have Popsicles                                                                                        0.000000
 Apple Magic (The Collector's series)                                                                         0.000000
 Ask Lily (Young Women of Faith: Lily Series, Book 5)                                                         8.000000
 Beyond IBM: Leadership Marketing and Finance for the 1990s                                                   0.000000
                                                                                                                ...   
Ã?Â?lpiraten.                                                                                                 0.000000
Ã?Â?rger mit Produkt X. Roman.                                                                                5.250000
Ã?Â?sterlich leben.                  

In [26]:
avg_rating_title = df.groupby(['book_title'])['book_rating'].mean().reset_index().rename(
    columns = {'book_rating' : 'avg_rating'})

In [27]:
popularity_df = pd.merge(count_rating_title, avg_rating_title, on = 'book_title', how = 'inner')

In [28]:
popularity_df.shape

(241071, 3)

- **Top 50 Books Sorted According to Average Rating**

In [29]:
popularity_df = popularity_df[popularity_df['no_of_ratings'] >= 250].sort_values(['avg_rating', 'no_of_ratings'],
                                                                                 ascending = False).head(50)

popularity_df

,book_title,no_of_ratings,avg_rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
219741,To Kill a Mockingbird,510,4.700000


- **Same Book has been rated by Multiple Users**

In [30]:
popularity_df = pd.merge(df, popularity_df, on = 'book_title', how = 'inner').sort_values(['avg_rating', 'no_of_ratings'],
                                                                          ascending = False).drop_duplicates(['book_title']).drop(columns = ['user_id', 'book_rating'])

In [31]:
popularity_df.shape

(50, 6)

In [32]:
popularity_df

,book_title,book_author,year_of_publication,image_url_m,no_of_ratings,avg_rating
17068,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
18906,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
15409,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
19293,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
16512,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16231,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
12315,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
13744,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
10128,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
0,To Kill a Mockingbird,Harper Lee,1988,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


<br>

<br>

<br>

### User Based Recommendation System

<br>

- **Rows : Book Titles**


- **Columns : Users**


- **Values : Ratings**

- **Number of times book is rated > 200**

- **There are duplicate Users as well as Books**

In [33]:
df['user_id'].duplicated().value_counts()

True     939030
False     92106
Name: user_id, dtype: int64

In [34]:
df['book_title'].duplicated().value_counts()

True     790065
False    241071
Name: book_title, dtype: int64

- **User Must be a Pro User**

- **User who had read & rated  more than 200 Books.**

- **No of books every user read & rated.**

In [35]:
# No of books every user read & rated

df.groupby(['user_id']).count()

,book_title,book_author,year_of_publication,image_url_m,book_rating
user_id,,,,,
2,1,1,1,1,1
8,17,17,17,17,17
9,3,3,3,3,3
10,1,1,1,1,1
12,1,1,1,1,1
...,...,...,...,...,...
278846,1,1,1,1,1
278849,4,4,4,4,4
278851,23,23,23,23,23


In [36]:
df.groupby(['user_id'])['book_rating'].count()

user_id
2          1
8         17
9          3
10         1
12         1
          ..
278846     1
278849     4
278851    23
278852     1
278854     8
Name: book_rating, Length: 92106, dtype: int64

- **Contains Both True & False**

In [37]:
df.groupby('user_id')['book_rating'].count() > 200

user_id
2         False
8         False
9         False
10        False
12        False
          ...  
278846    False
278849    False
278851    False
278852    False
278854    False
Name: book_rating, Length: 92106, dtype: bool

In [38]:
temp = df.groupby('user_id')['book_rating'].count() > 200
temp

user_id
2         False
8         False
9         False
10        False
12        False
          ...  
278846    False
278849    False
278851    False
278852    False
278854    False
Name: book_rating, Length: 92106, dtype: bool

In [39]:
(df.groupby('user_id')['book_rating'].count() > 200).size

# temp.size

92106

In [40]:
(df.groupby('user_id')['book_rating'].count() > 200).value_counts()

# temp.value_counts()

False    91295
True       811
Name: book_rating, dtype: int64

In [41]:
(df.groupby('user_id')['book_rating'].count() > 200).index

# temp.index

Int64Index([     2,      8,      9,     10,     12,     14,     16,     17,
                19,     20,
            ...
            278832, 278836, 278838, 278843, 278844, 278846, 278849, 278851,
            278852, 278854],
           dtype='int64', name='user_id', length=92106)

- **Contains Only True**

In [42]:
# User_ID for Pro Users

temp[temp]

user_id
254       True
2276      True
2766      True
2977      True
3363      True
          ... 
274308    True
275970    True
277427    True
277639    True
278418    True
Name: book_rating, Length: 811, dtype: bool

In [43]:
# No. of Pro Users

temp[temp].size

811

In [44]:
# User_ID for Pro Users

pro_user_id = temp[temp].index

In [45]:
pro_user_id

Int64Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,
              6323,   6543,
            ...
            271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
            277639, 278418],
           dtype='int64', name='user_id', length=811)

In [46]:
# No. of Pro Users

pro_user_id.size

811

In [47]:
# Data of Pro Users

df[df['user_id'].isin(pro_user_id)]

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
3,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,11676,8
6,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,85526,0
7,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,96054,0
10,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,177458,0
21,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,http://images.amazon.com/images/P/0374157065.0...,110912,10
...,...,...,...,...,...,...
1031124,Slow Food(The Case For Taste),Carlo Petrini,2003,http://images.amazon.com/images/P/0231128444.0...,275970,0
1031125,Strong Democracy : Participatory Politics for ...,Benjamin R. Barber,2004,http://images.amazon.com/images/P/0520242335.0...,275970,0
1031126,"Burpee Gardening Cyclopedia: A Concise, Up to ...",Allan Armitage,2002,http://images.amazon.com/images/P/0762412119.0...,275970,0
1031127,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,http://images.amazon.com/images/P/1582380805.0...,275970,0


In [48]:
df[df['user_id'] == 254]

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
6230,The Joy Luck Club,Amy Tan,1994,http://images.amazon.com/images/P/0804106304.0...,254,0
7478,The Catcher in the Rye,J.D. Salinger,1991,http://images.amazon.com/images/P/0316769487.0...,254,0
16078,Girl with a Pearl Earring,Tracy Chevalier,2001,http://images.amazon.com/images/P/0452282152.0...,254,0
18707,Night Watch,Terry Pratchett,2002,http://images.amazon.com/images/P/0060013117.0...,254,0
18722,Thief of Time,Terry Pratchett,2001,http://images.amazon.com/images/P/0060199563.0...,254,0
...,...,...,...,...,...,...
339264,Falling Angel (Rita Award),Anne Stuart,1995,http://images.amazon.com/images/P/0373600739.0...,254,0
339271,"Race To The Altar (Silhouette Special Edition,...",Patricia Hagan,2001,http://images.amazon.com/images/P/0373243979.0...,254,4
339274,Citizen of the Galaxy,ROBERT A. HEINLEIN,1987,http://images.amazon.com/images/P/0345342445.0...,254,0
339282,Star Beast,ROBERT A. HEINLEIN,1987,http://images.amazon.com/images/P/0345350596.0...,254,0


In [49]:
# Data of Pro Users

pro_users_df = df[df['user_id'].isin(pro_user_id)]
pro_users_df

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
3,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,11676,8
6,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,85526,0
7,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,96054,0
10,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,177458,0
21,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,http://images.amazon.com/images/P/0374157065.0...,110912,10
...,...,...,...,...,...,...
1031124,Slow Food(The Case For Taste),Carlo Petrini,2003,http://images.amazon.com/images/P/0231128444.0...,275970,0
1031125,Strong Democracy : Participatory Politics for ...,Benjamin R. Barber,2004,http://images.amazon.com/images/P/0520242335.0...,275970,0
1031126,"Burpee Gardening Cyclopedia: A Concise, Up to ...",Allan Armitage,2002,http://images.amazon.com/images/P/0762412119.0...,275970,0
1031127,Tropical Rainforests: 230 Species in Full Colo...,"Allen M., Ph.D. Young",2001,http://images.amazon.com/images/P/1582380805.0...,275970,0


- **No of Ratings for Every Book >= 50**

- **Every Book Should be Rated by at least 50 Different Users**

In [50]:
pro_users_df.groupby(['book_title']).count()

,book_author,year_of_publication,image_url_m,user_id,book_rating
book_title,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",2,2,2,2,2
Always Have Popsicles,1,1,1,1,1
Apple Magic (The Collector's series),1,1,1,1,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,1,1,1,1
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),1,1,1,1,1
...,...,...,...,...,...
Ã?Â?ber das Fernsehen.,2,2,2,2,2
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.,3,3,3,3,3
Ã?Â?lpiraten.,1,1,1,1,1


In [51]:
pro_users_df.groupby(['book_title']).count()['book_rating'].sort_values(ascending = False)

book_title
Wild Animus                                                                                     330
Bridget Jones's Diary                                                                           263
The Lovely Bones: A Novel                                                                       261
The Notebook                                                                                    238
The Pelican Brief                                                                               229
                                                                                               ... 
Pregnancy After a Loss: A Guide to Pregnancy After a Miscarriage, Stillbirth or Infant Death      1
Pregnancy &amp; childbirth: The complete guide for a new life                                     1
Preferential Policies: An International Perspective                                               1
Predicting the Future: From Jules Verne to Bill Gates                                    

In [52]:
pro_users_df['book_title'].value_counts()

Wild Animus                                                            330
Bridget Jones's Diary                                                  263
The Lovely Bones: A Novel                                              261
The Notebook                                                           238
The Pelican Brief                                                      229
                                                                      ... 
Engine City (The Engines of Light, Book 3)                               1
The Desperate Remedy: Henry Gresham and the Gunpowder Plot; A Novel      1
The Goddess Worldweaver (Seven Circles Trilogy, Book 3)                  1
Dying to Get Even (Jennifer Marsh Mysteries)                             1
Cocktail Classics                                                        1
Name: book_title, Length: 155845, dtype: int64

In [53]:
pro_users_df.groupby(['book_title']).count()['book_rating']

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    2
 Always Have Popsicles                                                                                        1
 Apple Magic (The Collector's series)                                                                         1
 Beyond IBM: Leadership Marketing and Finance for the 1990s                                                   1
 Clifford Visita El Hospital (Clifford El Gran Perro Colorado)                                                1
                                                                                                             ..
Ã?Â?ber das Fernsehen.                                                                                        2
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.                                                           3
Ã?Â?lpiraten.                                                                                

In [54]:
# Books those have been rated more than 50
# Both True/False

pro_users_df.groupby(['book_title']).count()['book_rating'] >= 50

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    False
 Always Have Popsicles                                                                                        False
 Apple Magic (The Collector's series)                                                                         False
 Beyond IBM: Leadership Marketing and Finance for the 1990s                                                   False
 Clifford Visita El Hospital (Clifford El Gran Perro Colorado)                                                False
                                                                                                              ...  
Ã?Â?ber das Fernsehen.                                                                                        False
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.                                                           False
Ã?Â?lpiraten.                                                

In [55]:
temp = pro_users_df.groupby(['book_title']).count()['book_rating'] >= 50
temp

book_title
 A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)    False
 Always Have Popsicles                                                                                        False
 Apple Magic (The Collector's series)                                                                         False
 Beyond IBM: Leadership Marketing and Finance for the 1990s                                                   False
 Clifford Visita El Hospital (Clifford El Gran Perro Colorado)                                                False
                                                                                                              ...  
Ã?Â?ber das Fernsehen.                                                                                        False
Ã?Â?ber die Pflicht zum Ungehorsam gegen den Staat.                                                           False
Ã?Â?lpiraten.                                                

In [56]:
temp.size

155845

In [57]:
temp.value_counts()

False    155139
True        706
Name: book_rating, dtype: int64

In [58]:
# Books those have been rated more than 50
# Only True

temp[temp]

book_title
1984                                                                 True
1st to Die: A Novel                                                  True
2nd Chance                                                           True
4 Blondes                                                            True
A Bend in the Road                                                   True
                                                                     ... 
Year of Wonders                                                      True
You Belong To Me                                                     True
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values    True
Zoya                                                                 True
\O\" Is for Outlaw"                                                  True
Name: book_rating, Length: 706, dtype: bool

In [59]:
temp[temp].value_counts()

True    706
Name: book_rating, dtype: int64

In [60]:
famous_book_names = temp[temp].index

In [61]:
famous_book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='book_title', length=706)

In [62]:
pro_users_df[pro_users_df['book_title'].isin(famous_book_names)]

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
31,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113519,0
...,...,...,...,...,...,...
1028414,Echoes,Nancy Morse,1992,http://images.amazon.com/images/P/1878702831.0...,238781,0
1028600,I Know Why the Caged Bird Sings,Maya Angelou,1996,http://images.amazon.com/images/P/0394429869.0...,239594,8
1028602,The Promise,CHAIM POTOK,1997,http://images.amazon.com/images/P/0449001164.0...,239594,7
1028815,The Pillars of the Earth,Ken Follett,2002,http://images.amazon.com/images/P/0743527631.0...,240144,0


In [63]:
filtered_df = pro_users_df[pro_users_df['book_title'].isin(famous_book_names)]

In [64]:
filtered_df

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
31,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113519,0
...,...,...,...,...,...,...
1028414,Echoes,Nancy Morse,1992,http://images.amazon.com/images/P/1878702831.0...,238781,0
1028600,I Know Why the Caged Bird Sings,Maya Angelou,1996,http://images.amazon.com/images/P/0394429869.0...,239594,8
1028602,The Promise,CHAIM POTOK,1997,http://images.amazon.com/images/P/0449001164.0...,239594,7
1028815,The Pillars of the Earth,Ken Follett,2002,http://images.amazon.com/images/P/0743527631.0...,240144,0


In [65]:
filtered_df

,book_title,book_author,year_of_publication,image_url_m,user_id,book_rating
31,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,The Kitchen God's Wife,Amy Tan,1991,http://images.amazon.com/images/P/0399135782.0...,113519,0
...,...,...,...,...,...,...
1028414,Echoes,Nancy Morse,1992,http://images.amazon.com/images/P/1878702831.0...,238781,0
1028600,I Know Why the Caged Bird Sings,Maya Angelou,1996,http://images.amazon.com/images/P/0394429869.0...,239594,8
1028602,The Promise,CHAIM POTOK,1997,http://images.amazon.com/images/P/0449001164.0...,239594,7
1028815,The Pillars of the Earth,Ken Follett,2002,http://images.amazon.com/images/P/0743527631.0...,240144,0


In [66]:
filtered_df['user_id'].duplicated().value_counts()

True     57776
False      810
Name: user_id, dtype: int64

In [67]:
filtered_df['book_title'].duplicated().value_counts()

True     57880
False      706
Name: book_title, dtype: int64

<br>

<br>

## Colaborative Recommendation System

<br>

In [68]:
cf = filtered_df.pivot_table(index = 'book_title', columns = 'user_id', values = 'book_rating')
cf

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [69]:
cf.fillna(0, inplace = True)
cf

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
cf.index.duplicated().sum()

0

In [71]:
cf.columns.duplicated().sum()

0

<br>

### Finding Similarity

- **Cosine Similarity between Every Book.**


- **Which 5 Users are most similar.**

In [72]:
similarity = cosine_similarity(cf)

In [73]:
similarity

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [74]:
similarity.shape

(706, 706)

In [75]:
cf.index[0]

'1984'

- **Getting Index of Movie**

In [76]:
title = '1984'

In [77]:
display(cf[cf.index == title])
display(cf[cf.index == title].index)
display(cf[cf.index == title].index[0])

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Index(['1984'], dtype='object', name='book_title')

'1984'

In [78]:
reset_index_df = cf.reset_index()
reset_index_df[reset_index_df['book_title'] == title].index
reset_index_df[reset_index_df['book_title'] == title].index[0]

0

In [79]:
# Similarity between 0th book and remaining all.

similarity[0]

array([1.        , 0.10255025, 0.01220856, 0.        , 0.05367224,
       0.02774901, 0.08216491, 0.13732869, 0.03261686, 0.03667591,
       0.02322418, 0.06766487, 0.02083978, 0.09673735, 0.13388865,
       0.08303112, 0.11153543, 0.05100411, 0.02517784, 0.11706383,
       0.        , 0.14333793, 0.07847534, 0.06150451, 0.08723968,
       0.        , 0.07009814, 0.13658681, 0.07600328, 0.12167134,
       0.00768046, 0.01473221, 0.        , 0.07965814, 0.04522617,
       0.01556271, 0.09495938, 0.0182307 , 0.02610465, 0.07984012,
       0.11679969, 0.0569124 , 0.08354155, 0.08471898, 0.08785938,
       0.05491435, 0.0548505 , 0.27026514, 0.09779123, 0.06016046,
       0.08958835, 0.06748675, 0.        , 0.04468098, 0.01920872,
       0.        , 0.05629067, 0.00557964, 0.07877059, 0.05219479,
       0.18908177, 0.        , 0.01240656, 0.02984572, 0.04279502,
       0.12680125, 0.16566735, 0.        , 0.13357242, 0.06615478,
       0.        , 0.        , 0.        , 0.10968075, 0.02806

In [80]:
similarity[0].shape

(706,)

In [81]:
list(enumerate(similarity[0]))

[(0, 0.9999999999999998),
 (1, 0.10255024829874465),
 (2, 0.012208555646509366),
 (3, 0.0),
 (4, 0.053672244225442485),
 (5, 0.027749013024662858),
 (6, 0.08216491130000354),
 (7, 0.13732869060263378),
 (8, 0.03261686350494277),
 (9, 0.03667590746512177),
 (10, 0.02322418425912186),
 (11, 0.06766486753662343),
 (12, 0.020839782065795794),
 (13, 0.09673734976610586),
 (14, 0.13388864701187034),
 (15, 0.08303111691811028),
 (16, 0.1115354275858341),
 (17, 0.051004105100675136),
 (18, 0.025177841632809915),
 (19, 0.11706383185253948),
 (20, 0.0),
 (21, 0.14333792815336935),
 (22, 0.07847534327232125),
 (23, 0.06150451339525138),
 (24, 0.08723968326958673),
 (25, 0.0),
 (26, 0.07009814362931202),
 (27, 0.13658680855472938),
 (28, 0.07600327535501016),
 (29, 0.12167133598776537),
 (30, 0.007680459111722776),
 (31, 0.01473221255132234),
 (32, 0.0),
 (33, 0.07965814289647455),
 (34, 0.04522616573040869),
 (35, 0.01556270914112474),
 (36, 0.09495937740802214),
 (37, 0.018230696811058135),
 (38

In [82]:
sorted(list(enumerate(similarity[0])), reverse = True)

[(705, 0.04316045784191231),
 (704, 0.07347566833900478),
 (703, 0.12110366748083307),
 (702, 0.017696441246693336),
 (701, 0.05864096359154978),
 (700, 0.009099054369341325),
 (699, 0.12530648498478347),
 (698, 0.009687245558530464),
 (697, 0.011401692977613393),
 (696, 0.05237226614377158),
 (695, 0.03734566872286401),
 (694, 0.016351262734144115),
 (693, 0.02407593195976403),
 (692, 0.07151793939760116),
 (691, 0.0),
 (690, 0.07785800873431292),
 (689, 0.0839284865111645),
 (688, 0.042188025333763236),
 (687, 0.0),
 (686, 0.08143881222401984),
 (685, 0.007305889033771798),
 (684, 0.09095705260744957),
 (683, 0.08418288167449578),
 (682, 0.1265536444037226),
 (681, 0.0),
 (680, 0.018394279624570017),
 (679, 0.0),
 (678, 0.020669978441948792),
 (677, 0.0672128392842741),
 (676, 0.0),
 (675, 0.057940637804840725),
 (674, 0.13365875731715243),
 (673, 0.009489432211152974),
 (672, 0.09488374038474037),
 (671, 0.05711356172333468),
 (670, 0.09993168845101595),
 (669, 0.0),
 (668, 0.010607

In [83]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])

[(0, 0.9999999999999998),
 (47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397),
 (551, 0.2262639743141286),
 (181, 0.22222254415660225),
 (500, 0.21057743765309012),
 (344, 0.20675540390454),
 (554, 0.20025063706783053),
 (520, 0.19972624710300058),
 (527, 0.19656287691544355),
 (488, 0.19511819825564608),
 (552, 0.1920889620576415),
 (318, 0.19182230688281934),
 (624, 0.18925313707609975),
 (598, 0.18921367782346202),
 (60, 0.18908176672188648),
 (622, 0.18571699439818884),
 (647, 0.1841081817659337),
 (546, 0.18408910419891056),
 (202, 0.18049207281616433),
 (541, 0.17990898085771032),
 (309, 0.17415730924332484),
 (588, 0.1738182944100774),
 (537, 0.17008719986228352),
 (544, 0.1672877700938274),
 (66, 0.16566734565340113),
 (118, 0.165272766123669),
 (269, 0.1650329228149958),
 (354, 0.16337845332530374),
 (476, 0.16104067376144765),
 (407, 0.16041182196659062),
 (590, 0.16000267935045498),
 (666, 0.1589387141616799),
 (92, 0

In [84]:
# Top 5 Similar Books 

sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])[1:6]

[(47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397),
 (551, 0.2262639743141286)]

In [85]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])[1:6][0]

(47, 0.2702651417103732)

In [86]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])[1:6][0][0]

47

In [87]:
cf.head(1)

user_id,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
cf.index[0]

'1984'

In [89]:
book_index = list()
book_names = list()

book_list = sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x : x[1])[1:6]

[book_index.append(i[0]) for i in book_list]
    
[book_names.append(cf.index[i]) for i in book_index]
    
display(book_index, book_names) 

[47, 545, 82, 634, 551]

['Animal Farm',
 "The Handmaid's Tale",
 'Brave New World',
 'The Vampire Lestat (Vampire Chronicles, Book II)',
 'The Hours : A Novel']

In [90]:
np.where(cf.index == title)

(array([0], dtype=int64),)

In [91]:
np.where(cf.index == title)[0]

array([0], dtype=int64)

In [92]:
np.where(cf.index == title)[0][0]

0

In [93]:
def recommend(title):
    index = np.where(cf.index == title)[0][0]
    similar_books = sorted(list(enumerate(similarity[index])), reverse = True, key = lambda x : x[1])[1:6]
    similar_books_indices = [i[0] for i in similar_books]
    
    data = list()
    
    for i in similar_books_indices:
        item = list()
        book_name = cf.index[i]
        temp_df = books[books['book_title'] == book_name]
        item.extend(list(temp_df.drop_duplicates('book_title')['book_title'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['book_author'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['image_url_m'].values))
        
        data.append(item)
    
    return data     

In [94]:
recommend('The Fellowship of the Ring (The Lord of the Rings, Part 1)')

[['The Two Towers (The Lord of the Rings, Part 2)',
  'J.R.R. TOLKIEN',
  'http://images.amazon.com/images/P/0345339711.01.MZZZZZZZ.jpg'],
 ['The Return of the King (The Lord of the Rings, Part 3)',
  'J.R.R. TOLKIEN',
  'http://images.amazon.com/images/P/0345339738.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Prisoner of Azkaban (Book 3)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Chamber of Secrets (Book 2)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439064872.01.MZZZZZZZ.jpg']]

In [95]:
display(count_rating_title.head(2), avg_rating_title.head(2))

,book_title,no_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1


,book_title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00


In [96]:
books = books.drop(columns = ['isbn', 'publisher', 'image_url_s', 'image_url_l'])

In [97]:
books = pd.merge(books, count_rating_title, on = 'book_title', how = 'inner')

In [98]:
books = pd.merge(books, avg_rating_title, on = 'book_title', how = 'inner')

In [99]:
books

,book_title,book_author,year_of_publication,image_url_m,no_of_ratings,avg_rating
0,Classical Mythology,Mark P. O. Morford,2002,http://images.amazon.com/images/P/0195153448.0...,2,3.500000
1,Classical Mythology,Mark P. O. Morford,1998,http://images.amazon.com/images/P/0801319536.0...,2,3.500000
2,Clara Callan,Richard Bruce Wright,2001,http://images.amazon.com/images/P/0002005018.0...,14,4.928571
3,Decision in Normandy,Carlo D'Este,1991,http://images.amazon.com/images/P/0060973129.0...,3,5.000000
4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,http://images.amazon.com/images/P/0374157065.0...,11,4.272727
...,...,...,...,...,...,...
270291,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,http://images.amazon.com/images/P/0449906736.0...,1,5.000000
270292,From One to One Hundred,Teri Sloat,1991,http://images.amazon.com/images/P/0525447644.0...,1,4.000000
270293,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,http://images.amazon.com/images/P/006008667X.0...,1,0.000000
270294,Republic (World's Classics),Plato,1996,http://images.amazon.com/images/P/0192126040.0...,1,0.000000


#### Dumpping Required Things

- **Dumping Datasets**


- popularity_df

- cf

- similarity

- books

In [100]:
# pickle.dump(popularity_df, open('popularity_df.pkl', 'wb'))

# pickle.dump(cf, open('cf.pkl', 'wb'))

# pickle.dump(similarity, open('similarity.pkl', 'wb'))

# pickle.dump(books, open('books.pkl', 'wb'))

In [101]:
# !pip freeze

In [102]:
# pip freeze

In [111]:
pickle.format_version

'4.0'